# Защита персональных данных клиентов

`Цель:` защитить данные клиентов страховой компании «N». 

`Задача:` разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.


`Описание данных:`
    - Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
    - Целевой признак: количество страховых выплат клиенту за последние 5 лет.
      
    
`План:`
- Загрузка и изучние данных
- Исследование вопросов:
      + Изменится ли качество линейной регрессии при умножении признаков  на обратимую матрицу?
      + Как связаны параметры линейной регрессии в исходной задаче и в преобразованной?
- Разработка алгоритма преобразования данных для решения задачи      
- Реализация алгоритма, проверка качества регрессии
    
`Метрика:` R2    
    

### Подключение библиотек

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from numpy import linalg as LA
from sklearn.metrics import r2_score
from numpy.linalg import inv

## Загрузка данных

In [5]:
insurance_data = pd.read_csv('/datasets/insurance.csv')

In [6]:
insurance_data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [7]:
insurance_data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [8]:
insurance_data.tail()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0
4999,1,28.0,40600.0,1,0


In [9]:
insurance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Кол-во значений по всем признакам одинаково, значит пропусков нет.

Проверим данные на дубликаты.

In [10]:
insurance_data.duplicated().sum()

153

In [12]:
insurance_data = insurance_data.drop_duplicates()
insurance_data.duplicated().sum()

0

In [13]:
insurance_data.corr()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
Пол,1.000000,0.001953,0.015456,-0.007315,0.011565
Возраст,0.001953,1.000000,-0.017386,-0.009064,0.654964
Зарплата,0.015456,-0.017386,1.000000,-0.031687,-0.013123
Члены семьи,-0.007315,-0.009064,-0.031687,1.000000,-0.039303
Страховые выплаты,0.011565,0.654964,-0.013123,-0.039303,1.000000


Заметная корреляция только между параметрами Возраст/Страховые выплаты, но не слишком выокая.

Разделим данные на тренировочную и тестовую выборки.

In [16]:
features = insurance_data.drop('Страховые выплаты',axis=1)
target = insurance_data['Страховые выплаты']

In [17]:
features.head()

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0


In [18]:
target.head()

0    0
1    1
2    0
3    0
4    0
Name: Страховые выплаты, dtype: int64

In [21]:
features.shape
#target.shape

(4847, 4)

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

###### Вопрос: Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится.

**Обоснование:** 

Понадобятся формулы:


$$
1. w = (X^T X)^{-1} X^T y
$$

$$
2. (A*B)^T = B^T*A^T
$$

$$
3. (A*B)^{-1} = B^{-1}*A^{-1}
$$

$$
4. A*A^{-1}=E
$$

$$
5.A*E=A
$$

Пусть C - обратимая матрица.

Необходимо выяснить изменится ли качество линейной регрессии.
Подставим в формулу (1) 
$$
X = XC
$$
Получим:
$$
w= ((XC)^T XC)^{-1} (XC)^T y
$$
Преобразуем выражение используя св-ва матриц:

1. По формуле 3:

$$
w= (XC)^{-1}(XC)^T)^{-1}(XC)^T y
$$


2. По формуле 4  $$((XC)^T)^{-1}(XC)^T=E$$  

3. По формуле 5
$$
w = C^{-1}X^{-1}y
$$

4. Умножим левую и правую часть на C:
$$
Cw=X^{-1}y
$$

5. Веремся к изначальной формуле и преобразуем ее по тем же св-вам:
$$
w = (X^T X)^{-1} X^T y = X^{-1}y
$$

Отсюда видим, что:

$$
w -> Cw
$$


Сравним предсказания до и после преобразования X = XC, введя новый вектор w':

$$
a = a'
$$
$$
Xw=X'w'
$$
$$
XX^{-1}y=XC C^{-1}X^{-1}y
$$
$$
y=y
$$

## Алгоритм преобразования

**Алгоритм**

1. Сгенерируем случайную квадратную матрицу С
2. Проверим ее на обратимость (определитель не равен нулю)
3. Умножим матрицу признаком на матрицу С

Чтобы матрицы можно было умножать размерность С должна быть равно 4х4.



**Обоснование**

Умножение на обратимую матрицу не влияет на качество линейной регрессии т.к. было показано в пункте 2.

$$
Xw=X'w'
$$

## Проверка алгоритма

###### Реализация

In [25]:
def encryption(features):
    features[0] = 1
    n = features.shape[1]
    #print(n)
    C = np.random.normal(0,1,(n, n))
    #print(C)
    print(LA.det(C) != 0)
    C_inv = inv(C)
    features_encription = features.values.dot(C)
    return C, C_inv,features_encription

In [27]:
C, C_inv,features_encription = encryption(features)

True


In [28]:
features_train, features_test, target_train, target_test = train_test_split(features_encription,target,test_size=0.25,random_state=12345)

In [30]:
# Обучим модель на зашифрованных данных и оценим качество модели

model = LinearRegression().fit(features_train, target_train)
predictions = model.predict(features_test)
print(r2_score(target_test, predictions))

0.4352599711889201


In [31]:
# Проверим что качество модели без шифрования не отличается 

model = LinearRegression().fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4249455028666801


###### Вывод: качество модели без шифрования не отличается от модели построенной на зашифрованных данных
